In [50]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool

from dotenv import load_dotenv

load_dotenv()

True

## Summarization middleware

In [4]:
@tool
def weather_tool(query: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {query}!"

@tool
def calculator_tool(query: str) -> str:
    """Calculate the result of a given query."""
    return f"The result of {query} is {eval(query)}"

In [ ]:
agent = create_agent(
    model=ChatGoogleGenerativeAI(model="gemini-2.0-flash"),
    tools=[weather_tool, calculator_tool],
    middleware=[
        SummarizationMiddleware(
            model=ChatGoogleGenerativeAI(model="gemini-2.0-flash"),
            max_tokens_before_summary=100,  # Trigger summarization at 100 tokens
            messages_to_keep=20,  # Keep last 20 messages after summary
            summary_prompt="Custom prompt for summarization...",  # Optional
        ),
    ],
)

In [7]:
res = agent.invoke({"messages": [{"role": "user", "content": "what is the weather in sf"}]})

In [8]:
res["messages"][-1].content

"It's always sunny in sf!"

In [9]:
res = agent.invoke({"messages": [{"role": "user", "content": "2 + 2"}]})
res["messages"][-1].content

'The answer is 4.'

In [10]:
res = agent.invoke({"messages": [{"role": "user", "content": "2 + 2 * 3"}]})
res["messages"][-1].content

'The answer is 8.'

## Human-in-the-loop middleware

In [13]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

In [14]:
@tool
def read_email_tool(email_id: str) -> str:
    """Read an email with the given ID."""
    return f"Read email with id {email_id}"


@tool
def send_email_tool(email: str) -> str:
	"""Send an email."""
	return f"Sent email: {email}"

In [55]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call

def human_in_the_loop(request, handler):
    print("Model is about to act:")
    print(request.messages)
    
    decision = input("Approve? (y/n/edit): ")
    if decision == "edit":
        request.messages[-1].content = input("Enter your revision: ")
    elif decision == "n":
        print("Action canceled by human.")
        return None

    # Continue to next step (calls the actual model)
    return handler(request)


agent = create_agent(
    model=ChatGoogleGenerativeAI(model="gemini-2.0-flash"),
    tools=[read_email_tool, send_email_tool],
    middleware=[wrap_model_call(human_in_the_loop)]
)


In [58]:
res = agent.invoke(
	{"messages": [{"role": "user", "content": "send email 123"}]}, 
	{"configurable" : {"thread_id": "1"}},
)

Model is about to act:
[HumanMessage(content='send email 123', additional_kwargs={}, response_metadata={}, id='cf0c1fa3-ca83-4e04-984a-9f29174a65bc')]


In [59]:
res

{'messages': [HumanMessage(content='send email 123', additional_kwargs={}, response_metadata={}, id='cf0c1fa3-ca83-4e04-984a-9f29174a65bc'),
  AIMessage(content='I can help you send an email, but I need the content of the email first. Could you please provide the content?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--a8e81021-857c-485f-967a-a09823f62bec-0', usage_metadata={'input_tokens': 40, 'output_tokens': 26, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}})]}

## To-do-List middleware

In [33]:
from langchain.agents import create_agent
from langchain.agents.middleware import TodoListMiddleware
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool

from dotenv import load_dotenv

load_dotenv()

True

In [31]:
@tool
def read_file(file_path: str) -> str:
    """Read contents of a file."""
    with open(file_path) as f:
        return f.read()


@tool
def write_file(file_path: str, content: str) -> str:
    """Write content to a file."""
    with open(file_path, 'w') as f:
        f.write(content)
    return f"Wrote {len(content)} characters to {file_path}"


@tool
def run_tests(test_path: str) -> str:
    """Run tests and return results."""
    # Simplified for example
    return "All tests passed!"

In [ ]:
agent = create_agent(
    model=ChatGoogleGenerativeAI(model="gemini-2.0-flash"),
    tools=[read_file, write_file, run_tests],
    middleware=[TodoListMiddleware()],
)

In [38]:
result = agent.invoke({
    "messages": [HumanMessage("Refactor the authentication module to use async/await and ensure all tests pass")]
})

In [39]:
result["todos"]

[{'content': 'Refactor the authentication module to use async/await.',
  'status': 'completed'},
 {'content': 'Run tests for the authentication module.',
  'status': 'completed'},
 {'content': 'Ensure all tests pass.', 'status': 'completed'}]

In [40]:
result

{'messages': [HumanMessage(content='Refactor the authentication module to use async/await and ensure all tests pass', additional_kwargs={}, response_metadata={}, id='1c3abbb6-557d-4180-838f-514aa1829bc6'),
  AIMessage(content='Okay, I will refactor the authentication module to use async/await and ensure all tests pass.', additional_kwargs={'function_call': {'name': 'write_todos', 'arguments': '{"todos": [{"status": "in_progress", "content": "Refactor the authentication module to use async/await."}, {"status": "pending", "content": "Run tests for the authentication module."}, {"status": "pending", "content": "Ensure all tests pass."}]}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--a5d59b9d-8332-4d5b-af7a-60af1db13f2d-0', tool_calls=[{'name': 'write_todos', 'args': {'todos': [{'status': 'in_progres